<a href="https://colab.research.google.com/github/chloelbrt/Research_internship_examples/blob/main/Rapport_MI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Learning Tutorial of Gianni Franchi
PAN Herve - LEBRET Chloé - PAPADOPOULOS Robin 

Welcome to ML project!
**In this notebook, you will**:
- Learn what is SSL
- Learn the difficulty with Overfitting
- Learn to implement an Convolutional Neural Network.
- Learn to train it when we don't have enough data

If you have never used jupyter notebooks, nor Colab notebooks, [here](https://colab.research.google.com/notebooks/welcome.ipynb) is a short intro.






In [ ]:
import copy
from numpy import asarray
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
import random
import math
from torch.autograd import Variable
#import matplotlib.pyplot as plt
#from modules import *
#import torchvision.models as models_pytorch
#import h5py
#import torch.optim as optim
#import augmentations
from torch.nn.functional import kl_div, softmax, log_softmax
import torch.optim as optim
import torch.utils.data as data
from os.path import exists, join, split
from os import listdir
from os.path import join
from PIL import Image, ImageFilter , ImageDraw
import PIL
import random
#import madgrad 
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#! pip install madgrad
#! pip install efficientnet_pytorch

In [ ]:
num_class = 10 #number of classes
num_classes=num_class
seed=111 #seed for the algorithm
batch_size = 32
num_train =100 # number of training image by classe
cutout=16  # parameter for the cutout
num_epochs=50
#Validation set size
valid_size = 200

#An array with the different learning rates.

lrarray=[0.001, 0.01, 0.1, 0.5]


# First let us define a CNN

In [ ]:
bn_momentum = 0.9


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)


def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)


class WideBasic(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1):
        super(WideBasic, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=bn_momentum)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(planes, momentum=bn_momentum)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)

        return out


class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(WideResNet, self).__init__()
        self.in_planes = 16

        assert ((depth - 4) % 6 == 0), 'Wide-resnet depth should be 6n+4'
        n = int((depth - 4) / 6)
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3, nStages[0])
        self.layer1 = self._wide_layer(WideBasic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(WideBasic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(WideBasic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=bn_momentum)
        self.linear = nn.Linear(nStages[3], num_classes)

        # self.apply(conv_init)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        # out = F.avg_pool2d(out, 8)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out

# now let us define a dataset


In [ ]:
class Dataset_sub_CIFAR(data.Dataset):

    def __init__(self, data_feature, data_target,transform,phase='label'):
        self.data_feature = data_feature
        self.data_target = data_target
        self.transform = transform
        self.phase=phase

    def __len__(self):
        return len(self.data_feature)

    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        if self.phase=='label':
            data_feature = self.transform(Image.fromarray(np.uint8(self.data_feature[index])))
            data_target =  self.data_target[index]
            return data_feature, data_target

        else:
            data_feature = self.data_feature[index].float()
            return data_feature


class CutoutDefault(object):
    """
    Reference : https://github.com/quark0/darts/blob/master/cnn/utils.py
    """
    def __init__(self, length):
        self.length = length

    def __call__(self, img):
        if self.length <= 0:
            return img
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)
        y = np.random.randint(h)
        x = np.random.randint(w)

        y1 = np.clip(y - self.length // 2, 0, h)
        y2 = np.clip(y + self.length // 2, 0, h)
        x1 = np.clip(x - self.length // 2, 0, w)
        x2 = np.clip(x + self.length // 2, 0, w)

        mask[y1: y2, x1: x2] = 0.
        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img *= mask
        return img

    
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode = 'reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
    CutoutDefault(cutout),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])


############################## CHOISIR entre les deux data augmentations
res_size=224
transform_train = transforms.Compose([
    transforms.Resize([res_size, res_size]),
    transforms.RandomCrop(224, padding=4, padding_mode = 'reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
    CutoutDefault(cutout),
])

 

transform_test = transforms.Compose([
    transforms.Resize([res_size, res_size]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])

#Dataset loading
CIFAR10_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=None, download=True)
CIFAR10_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=None, download=True)

#Dataset for Resnet Alexnet WideResnet etc..

np.random.seed(seed=seed)
permuation=np.random.permutation(len(CIFAR10_train_dataset.targets))

Original_train_data_x = (CIFAR10_train_dataset.data)
Original_train_data_y = np.array(CIFAR10_train_dataset.targets)
Original_train_data_x = Original_train_data_x[permuation]
Original_train_data_y = Original_train_data_y[permuation]

Original_test_data_x = CIFAR10_test_dataset.data
Original_test_data_y = np.array(CIFAR10_test_dataset.targets)

#Selection of 250 labeled images for training and 2000 for validation
incr_class = torch.zeros(num_class)
train_idx_dico = {} #labeled images index dictionnary

for i in range(num_class):
    train_idx_dico[str(i)] = []

valid_idx = np.zeros(num_class * valid_size, dtype=np.int32) #validation images indexes (2000)
incr_t = 0
incr_v = 0
incrtotal = 0

for idx in range(len(Original_train_data_y)):
    class_y = Original_train_data_y[idx]
    incrtotal += 1

    train_idx_dico[str(class_y)].append(idx)
    incr_class[class_y] += 1 #count the number of image per class
    incr_t += 1


train_idx = np.zeros(num_class * num_train, dtype=np.int32) #train labeled images indexes (1000)
list_train_id = []
list_unalabel_id = []
valid_idx = []
unlabel_idx_dico = {}
for i in range(num_class):
    unlabel_idx_dico[str(i)] = []
for i in range(num_class):
    list_train_id = list_train_id + train_idx_dico[str(i)][0:num_train]
    valid_idx =valid_idx + train_idx_dico[str(i)][num_train:num_train+valid_size]
    list_unalabel_id = list_unalabel_id + train_idx_dico[str(i)][num_train+valid_size::]
    unlabel_idx_dico[str(i)] = train_idx_dico[str(i)][num_train::]

#Get labeled and unlabeled data

x_train = Original_train_data_x[[int(i) for i in list_train_id]]
y_train = Original_train_data_y[[int(i) for i in list_train_id]]

x_unlabeled = Original_train_data_x[[int(i) for i in list_unalabel_id]]
y_unlabeled = Original_train_data_y[[int(i) for i in list_unalabel_id]]

#Get validation set data
x_valid = Original_train_data_x[[int(i) for i in valid_idx]]
y_valid = Original_train_data_y[[int(i) for i in valid_idx]]

# Printing the size of the training, validation and test sets
print('Number of training examples: ' + str(x_train.shape[0]))
print('Number of unlabeled examples: ' + str(x_unlabeled.shape[0]))
print('Number of validation examples: ' + str(x_valid.shape[0]))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



#Dataloader creation

test_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(Original_test_data_x, Original_test_data_y, transform=transform_test),
    batch_size = batch_size,
    shuffle=False, num_workers=2)

train_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_train, y_train, transform=transform_train),
    batch_size=batch_size,shuffle=True, num_workers=2) #num_workers = 2 ou 1

valid_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_valid, y_valid, transform=transform_test),
    batch_size=batch_size,
    shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
#We are going to use the scikit-learn library to use their SVC and Random forest model.
#To train the model, we need to arrange the dataset for them (they use numpy arrays in their model)


X_valid, y_valid = [], []
for data, labels in valid_loader:
    X_valid.append(data.numpy())
    y_valid.append(labels.numpy())
X_valid = np.concatenate(X_valid)
y_valid = np.concatenate(y_valid)

X_valid = X_valid.reshape(X_valid.shape[0], -1)


X_train, y_train = [], []
for data, labels in train_loader:
    X_train.append(data.numpy())
    y_train.append(labels.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

X_train = X_train.reshape(X_train.shape[0], -1)



# Now we build the CNN and the optimizer

We will try different deep convolutional model to train with the CIFAR10 dataset :
WideResNet, Resnet, AlexNet 
(cuda shifts the process on the GPU)
You can also choose to work with a Random Forest or a SVC 


In [ ]:
#Launch this cell to work with WideResNet as the network

net = WideResNet(28, 2, dropout_rate=0.0, num_classes=num_class)
net =net.cuda()
net_save = WideResNet(28, 2, dropout_rate=0.0, num_classes=num_class) # model where to save the results
net_save =net_save.cuda()

In [ ]:
#Launch this cell to work with resnet18 as the network
#By adding the parameter weights="IMAGENET1K_V1" we use the pretrained weights, if we don't put anything for the parameter it starts from random weights
weights="IMAGENET1K_V1"
# weights=None

net = models.resnet18(weights=weights)
net.fc = nn.Linear(512, num_classes)
net = net.cuda()
net_save = models.resnet18(weights=weights)
net_save.fc = nn.Linear(512, num_classes)
net_save =net_save.cuda()


In [ ]:
#Launch this cell to work with alexnet as the network
#By adding the parameter weights="IMAGENET1K_V1" we use the pretrained weights, if you want to use random weight change the weights variable to None

weights="IMAGENET1K_V1"
# weights=None


net = models.alexnet(weights=weights)
net.classifier[6] = nn.Linear(4096,num_classes)
net = net.cuda()

net_save = models.alexnet(weights=weights)
net_save.classifier[6] = nn.Linear(4096,num_classes)
net_save =net_save.cuda()


In [ ]:
#Launch this cell to work with a Random Forest as the network
#You can modify the number of trees in the forest by changing the n_estimators parameter 
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
#Launch this cell to work with a SVC model as the network
clf = svm.SVC()

# Training

In [ ]:

criterion = nn.CrossEntropyLoss()
def learning_rate_scheduler(init, epoch):
    optim_factor = 0
    if(epoch > 200):
        optim_factor = 3
    elif(epoch > 160):
        optim_factor = 2
    elif(epoch > 80):
        optim_factor = 1

    return init*math.pow(0.1, optim_factor)


# Training 
def train(epoch,net,trainloader,log_interval=15):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate_scheduler(lr, epoch)))
    optimizer = optim.SGD(net.parameters(), lr=learning_rate_scheduler(lr, epoch), momentum=0.9, weight_decay=5e-4)
    for batch_idx, (inputs, targets) in enumerate(trainloader):
 
        inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        #inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx % log_interval == 0:
            print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainloader.dataset)//batch_size)+1, loss.item(), 100.*correct/total))


def test(epoch,net,testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))
    return acc

In [ ]:
#Training for the CNN models

best_acc=0
for epoch in range(num_epochs):

    train(epoch,net,train_loader)
    acc = test(epoch,net,valid_loader)
    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
        net_save.load_state_dict(net.state_dict(), strict=True)
        best_acc=acc



In [ ]:
#Training for the SVM or Random Forest
clf.fit(X_train, y_train)

# Evaluation

In [ ]:
#Evaluation for the CNN models 

import pandas as pd


def test_final(net,testloader):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()
            if batch_idx == 0:
                predicted_concat = predicted.clone()
            else:
                predicted_concat = torch.cat((predicted_concat, predicted), 0)

        # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| TEST \t\t\tLoss: %.4f Acc@1: %.2f%%" %( loss.item(), acc))
    return predicted_concat.cpu().numpy()
    

predicted_concat = test_final(net,test_loader)


id_concat =range(len(predicted_concat))
my_submission = pd.DataFrame({'Id': id_concat,'Expected': predicted_concat})

# you could use any filename. We choose submission here
my_submission.to_csv('submission2.csv', index=False)
print('we have saved the submission !! ')



In [ ]:
# Predict on the test data for the Random Forest or the SVM 
y_pred = clf.predict(X_valid)

# Calculate accuracy
acc = accuracy_score(y_valid, y_pred)
print("Accuracy:", acc)


Accuracy: 0.334


#FixMatch

In [ ]:

n_classes = 10
batch_size = 32
epochs = 50
train_class_size = 100
valid_class_size = 200
unlabeled_ratio = 7
confidence_threshold = 0.6

#set sizes
valid_size = n_classes * valid_class_size
labeled_size = n_classes * train_class_size
unlabeled_size = labeled_size * unlabeled_ratio

#load datasets
print("Loading datasets...")
train_ds = torchvision.datasets.CIFAR10(root="./cifar10", train=True,  download=True)
valid_ds = torchvision.datasets.CIFAR10(root="./cifar10", train=False, download=True)


#select random data
print("Generating permutations...")
rng = np.random.default_rng()
valid_ids     = rng.permutation(valid_ds.data.shape[0])[:valid_size].astype(int)
labeled_ids   = rng.permutation(train_ds.data.shape[0])[:labeled_size].astype(int)
unlabeled_ids = rng.permutation(train_ds.data.shape[0])[:unlabeled_size].astype(int)

#extract validation and training sets from datasets
print("Extracting datasets...")
x_valid = np.array(valid_ds.data)[valid_ids]
y_valid = np.array(valid_ds.targets)[valid_ids]
x_labeled = np.array(train_ds.data)[labeled_ids]
y_labeled = np.array(train_ds.targets)[labeled_ids]
x_unlabeled = np.array(train_ds.data)[unlabeled_ids]

#creating dataloaders
print("Creating dataloaders...")
test_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(Original_test_data_x, Original_test_data_y, transform=transform_test),
    batch_size = batch_size,shuffle=False, num_workers=2)
labeled_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_labeled, y_labeled, transform=transform_train),
    batch_size=batch_size,shuffle=True, num_workers=2)

Loading datasets...


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Generating permutations...
Extracting datasets...
Creating dataloaders...


In [ ]:
# Training 
def training_epoch(epoch,net,loader,log_interval=15):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate_scheduler(lr, epoch)))
    optimizer = optim.SGD(net.parameters(), lr=learning_rate_scheduler(lr, epoch), momentum=0.9, weight_decay=5e-4)
    for batch_idx, (inputs, targets) in enumerate(loader):
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx % log_interval == 0:
            print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainloader.dataset)//batch_size)+1, loss.item(), 100.*correct/total))


def test(epoch,net,testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))
    return acc

def learning_rate_scheduler(init, epoch):
    optim_factor = 0
    if(epoch > 200):
        optim_factor = 3
    elif(epoch > 160):
        optim_factor = 2
    elif(epoch > 80):
        optim_factor = 1
    return init*math.pow(0.1, optim_factor)

def pseudo_label(net, unlabeled, threshold=0.5):
  n = unlabeled.size(0)
  labels = np.zeros(n)
  label_count = 0
  predictions = net(unlabeled)
  for i in range(n):
    probs = torch.functional.softmax(predictions[i])
    confidence = torch.max(prob)
    if confidence > threshold:
      labels[i] = [1 if p == confidence else 0 for p in prob]
  return labels


def pseudolabeled_loader(unlabeled, labels):
  count = torch.count_nonzero(labels)
  xy_index = 0
  nonzeros = torch.nonzero(torch.sum(labels,dim=2))
  n = nonzeros.size(0)
  x = torch.zeros((n,*unlabeled[0].size))
  y = torch.zeros((n,n_classes))
  for i in nonzeros:
    x[xy_index] = unlabeled[i]
    y[xy_index] = labels[i]
  return torch.utils.data.DataLoader(Dataset_sub_CIFAR(x, y), batch_size = batch_size,shuffle=False, num_workers=2)



In [ ]:
net = WideResNet(28, 2, dropout_rate=0.0, num_classes=num_class)
net = net.cuda()
net_save = WideResNet(28, 2, dropout_rate=0.0, num_classes=num_class) # model where to save the results
net_save = net_save.cuda()

criterion = nn.CrossEntropyLoss()

weak_augment = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(32),
    transforms.RandomCrop(size=32,padding=int(32*0.125),padding_mode='reflect')
])

def strong_augment(n_augments=2,magnitude=0.5):
  return transforms.Compose([
      transforms.RandomErasing(p=1,ratio=(1,1),scale=(0.1,0.1),value=127),
      transforms.RandAugment(n_augments,magnitude) ])

#Labeled training
net.train()
for epoch in range(epochs):
  training_epoch(epoch, net, labeled_loader)

for epoch in range(epochs):
  weak_augments = weak_augment(x_unlabeled)
  pseudolabels = pseudo_label(net, weak_augments, 0.6)
  strong_augments = strong_augment()(x_unlabeled)
  loader = pseudolabeled_loader(strong_augments, pseudolabels)
  training_epoch(epoch, net, pseudolabeled_loader)




RuntimeError: ignored

# Question for the report
I want that you send me a small report with the answer to this question and your notebook.
- Q0: Please train wideresnet, and please understand a bit wideresnet.
- Q1: Please change DNN with a Resnet 18. Try with one that is pre-trained and one that is not pre-trained. 
- Q2: Please change DNN with an AlexNet. Try with one that is pre-trained and one that is not pre-trained. (Be careful, you need a bit to play with the learning rate, for questions two, one and zeros I want to see the training loss and training accuracy. What other curb is interesting? Plot it and analyse it.)
- Q3: Please try to train an SVM and a random forest.
- Q4 After you have trained several models please draw a table and make some conclusions.
-  Q5 Read the paper fixmatch (https://amitness.com/2020/03/fixmatch-semi-supervised/) and explain it.
- Q6 please try to implement it and try to make it work.
- Q7 What can we do to avoid overfitting in Deep learning?


*Q0-Q5 = 14 pts*

*Q6 = 6 pts*

*Q7 = 1 pts*